In [1]:
import os, requests
import json, uuid
import pandas as pd
import geopandas as gpd
import traceback
import zipfile
import base64

## Descargar geometría

In [30]:
nombreSHP = "COLOMBIA/GPT_CNE"
url = 'http://dhime.ideam.gov.co/arcgis/rest/services/CNE/CatalogoNacionalEstaciones/MapServer/0/query'

In [33]:
class RequestDataAGOL(object):
    def __init__(self):
        self.url = url
        self.listaOids = []
        
    def countOnly(self):
        response = requests.post(
            self.url,
            data = {
            'where': '1=1',
            'outFields': "*",
            'returnCountOnly': 'true',
            'f': 'pjson'
            }
        )
        res = json.loads(response.text)
        self.cantidad = res["count"]
        print(self.cantidad)

    def oidsOnly(self):
        response = requests.post(
            self.url,
            data = {
            'where': "1=1",
            'outFields': "*",
            'returnIdsOnly': 'true',
            'f': 'pjson'
            }
        )
        res = json.loads(response.text)
        self.listaOids.extend(res["objectIds"])
        self.listaOids = list(set(self.listaOids))

    def requestData(self):
        global nombreSHP
        data = self.listaOids
        chunks = [data[x:x+1000] for x in range(0, len(data), 1000)]
        geo_df = ""
        a = 0
        for n in chunks:
            a += 1
            try:
                # print(len(n))
                n = ','.join([str(x) for x in n])
                response = requests.post(
                    self.url,
                    data = {
                    'objectIds': n,
                    'outFields': "*",
                    'f': 'geojson'
                })
                res = json.loads(response.text)
                gdf = gpd.GeoDataFrame.from_features(res['features'])
                gdf.crs = {'init' :'epsg:4326'}
                if type(geo_df) == str:
                    geo_df = gdf[:]
                else:
                    geo_df = geo_df.append(gdf)
                print("geo_df tiene {} elementos".format(len(geo_df)))
                geo_df.to_file(driver="ESRI Shapefile", filename="{}.shp".format(nombreSHP))
            except Exception:
                nombreSHP = "{}_{}".format(nombreSHP, a)
                print(traceback.print_exc())
    def main(self):
        self.countOnly()
        self.oidsOnly()
        self.requestData()

In [34]:
poo = RequestDataAGOL()
poo.main()

8906
geo_df tiene 1000 elementos
geo_df tiene 2000 elementos
geo_df tiene 3000 elementos
geo_df tiene 4000 elementos
geo_df tiene 5000 elementos
geo_df tiene 6000 elementos
geo_df tiene 7000 elementos
geo_df tiene 8000 elementos
geo_df tiene 8906 elementos


In [19]:
gdf = gpd.read_file("COLOMBIA/GPT_CO_CNE.shp")
gdf

,objectid,idestacion,nombre,idcategori,idtecnolog,idtipotran,idestadoes,fechainsta,altitud,latitud,...,fechasuspe,idaquarius,idgis,prop_nom,prop_cod,prop_tipo,prop_secto,temp,eliminado,geometry
0,67647,21208080,MONDONEDO_TEST [21208080],LG,TEST003,TTAS004,ESTA001,4.053960e+10,0,4.000000,...,NaN,429b0385-6d28-4124-a6d1-af438faa538b,None,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,84,Monitoreo Ambiental Permanente,Privado,Monitoreo Ambiental Permanente,0,POINT (-74.00000 4.00000)
1,67648,21209080,CAFAM LA FLORESTA [21209080],LM,TEST003,TTAS004,ESTA001,6.533748e+11,2671,4.683333,...,NaN,None,None,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,84,Monitoreo Ambiental Permanente,Privado,Monitoreo Ambiental Permanente,0,POINT (-74.06667 4.68333)
2,67649,21209420,USAQUEN GARITA [21209420],LM,TEST003,TTAS004,ESTA001,6.112116e+11,2620,4.683333,...,NaN,None,None,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,84,Monitoreo Ambiental Permanente,Privado,Monitoreo Ambiental Permanente,0,POINT (-74.08333 4.68333)
3,67650,21209460,PTE FRANCIS [21209460],LM,TEST003,TTAS004,ESTA001,5.480532e+11,3400,4.683333,...,NaN,None,None,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,84,Monitoreo Ambiental Permanente,Privado,Monitoreo Ambiental Permanente,0,POINT (-73.98333 4.68333)
4,67651,21208240,GIRARDOT [21208240],LG,TEST003,TTAS004,ESTA001,7.743240e+10,257,4.316667,...,NaN,None,None,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,84,Monitoreo Ambiental Permanente,Privado,Monitoreo Ambiental Permanente,0,POINT (-74.76667 4.31667)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8901,94699,3507500146,GARAGOA CESAM [35075110],CP,TEST001,TTAS002,ESTA001,1.517443e+12,1747,5.081347,...,NaN,None,None,CORPORACIÃN AUTÃNOMA REGIONAL DE CHIVOR,322,Autoridad Ambiental,PÃºblico,None,0,POINT (-73.36004 5.08135)
8902,95095,3507500142,GARAGOA CESAM [35075110],CP,TEST001,TTAS002,ESTA001,1.517443e+12,1747,5.081347,...,NaN,None,None,CORPORACIÃN AUTÃNOMA REGIONAL DE CHIVOR,322,Autoridad Ambiental,PÃºblico,None,0,POINT (-73.36004 5.08135)
8903,95495,5311500147,INFANTERIA MARINA,ME,TEST001,TTAS002,ESTA001,1.576195e+12,7,3.890083,...,NaN,None,None,ESTABLECIMIENTO PUBLICO AMBIENTAL DISTRITO DE ...,2482,Autoridad Ambiental,PÃºblico,None,0,POINT (-77.07033 3.89008)
8904,95895,5311500148,COLEGIO VNB,ME,TEST001,TTAS002,ESTA001,1.576195e+12,9,3.884194,...,NaN,None,None,ESTABLECIMIENTO PUBLICO AMBIENTAL DISTRITO DE ...,2482,Autoridad Ambiental,PÃºblico,None,0,POINT (-77.04938 3.88419)


In [20]:
gdf.columns

Index(['objectid', 'idestacion', 'nombre', 'idcategori', 'idtecnolog',
       'idtipotran', 'idestadoes', 'fechainsta', 'altitud', 'latitud',
       'longitud', 'iddepartam', 'idmunicipi', 'idareaoper', 'idareahidr',
       'idzonahidr', 'idsubzonah', 'familia', 'clase', 'codigowmo',
       'codigointe', 'codigoiata', 'codigonesd', 'codigooaci', 'identidad',
       'usuario', 'fechaultim', 'codigoorfe', 'observacio', 'idcorrient',
       'fechasuspe', 'idaquarius', 'idgis', 'prop_nom', 'prop_cod',
       'prop_tipo', 'prop_secto', 'temp', 'eliminado', 'geometry'],
      dtype='object')

## Descargar data

In [75]:
url = "http://dhime.ideam.gov.co/arcgis/rest/services/AtencionCiudadano/DescargarArchivo/GPServer/DescargarArchivo"

def get_data_colombia(parametro, variable, idestacion):
    '''
    ::params::
    parametro: PRECIPITACION, TEMPERATURA
    idestacion: codigo de estacion
    
    ::return::
    response: resultado de la consulta
    '''
    url_submit = os.path.join(url, "submitJob")
    params = {
        "Filtro": "sort=&filter=((IdParametro~eq~'%s'~and~Etiqueta~eq~'%s'~and~IdEstacion~eq~'%s'))&group=&fechaInicio=2000-01-01&fechaFin=2019-12-26&mostrarGrado=true&mostrarCalificador=true&mostrarNivelAprobacion=true"%(parametro, variable, idestacion),
        "Items": '[{"IdParametro":"%s","Etiqueta":"%s","EsEjeY1":false,"EsEjeY2":false,"EsTipoLinea":false,"EsTipoBarra":false,"TipoSerie":"Estandard","Calculo":"NA"}]'%(parametro, variable),
        "f": "json"
    }

    response = requests.post(url_submit, params)
    print(response.text)
    return response

In [72]:
def load_response(response):
    '''
    ::params::
    response: resultado del get_data
    
    ::return::
    filezip: archivo zip codificado
    '''
    jobid = json.loads(response.text)["jobId"]
    url_result = os.path.join(url, "jobs", jobid) # change jobid
    print(url_result)
    resp = requests.post(os.path.join(url_result, "results/Archivo"), {"f":"json"})
    print(resp.text)
    filezip = json.loads(resp.text)["value"]
    return filezip

In [73]:
def load_data(filezip):
    '''
    ::params::
    filezip: archivo zip codificado
    
    ::return::
    df: dataframe
    '''
    filezip_decode = base64.b64decode(filezip)
    f = open("test2.zip", "wb")
    f.write(filezip_decode)
    f.close()
    
    # reading zip file
    archive = zipfile.ZipFile('test2.zip', 'r')
    # decoding data
    data = archive.read('excel.csv.csv').decode("utf-8") 
    # sorting data
    datos = [x.split(',') for x in data.split('\n')]
    df = pd.DataFrame(datos[1:], columns=datos[0])
    return df

In [69]:
def process():
    response_data = get_data_colombia("PRECIPITACION", "24035410")
    file = load_response(response_data)
    data = load_data(file)
    return data

In [79]:
response_data = get_data_colombia("PRECIPITACION", "PTPM_TT_D", "37055010")

{"jobId":"j435b0f38c69f433d9498a257039c2715","jobStatus":"esriJobSubmitted"}


In [80]:
file = load_response(response_data)

http://dhime.ideam.gov.co/arcgis/rest/services/AtencionCiudadano/DescargarArchivo/GPServer/DescargarArchivo\jobs\j435b0f38c69f433d9498a257039c2715
{"paramName":"Archivo","dataType":"GPString","value":"..."}


In [81]:
data = load_data(file)

In [82]:
data

,CodigoEstacion,NombreEstacion,Latitud,Longitud,Altitud,Categoria,Entidad,AreaOperativa,Departamento,Municipio,...,FechaSuspension,IdParametro,Etiqueta,DescripcionSerie,Frecuencia,Fecha,Valor,Grado,Calificador,NivelAprobacion\r
0,37055010,AEROPUERTO SANTIAGO PÉREZ [37055010],7.0694444440,-70.73805556,128,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 08 - Santanderes-Arauca,Arauca,Arauca,...,,PRECIPITACION,PTPM_TT_D,Precipitación dia pluviométrico,Diaria,2000-01-01 00:00,0,50,,900\r
1,37055010,AEROPUERTO SANTIAGO PÉREZ [37055010],7.0694444440,-70.73805556,128,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 08 - Santanderes-Arauca,Arauca,Arauca,...,,PRECIPITACION,PTPM_TT_D,Precipitación dia pluviométrico,Diaria,2000-01-02 00:00,0,50,,900\r
2,37055010,AEROPUERTO SANTIAGO PÉREZ [37055010],7.0694444440,-70.73805556,128,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 08 - Santanderes-Arauca,Arauca,Arauca,...,,PRECIPITACION,PTPM_TT_D,Precipitación dia pluviométrico,Diaria,2000-01-03 00:00,0,50,,900\r
3,37055010,AEROPUERTO SANTIAGO PÉREZ [37055010],7.0694444440,-70.73805556,128,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 08 - Santanderes-Arauca,Arauca,Arauca,...,,PRECIPITACION,PTPM_TT_D,Precipitación dia pluviométrico,Diaria,2000-01-04 00:00,0,50,,900\r
4,37055010,AEROPUERTO SANTIAGO PÉREZ [37055010],7.0694444440,-70.73805556,128,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 08 - Santanderes-Arauca,Arauca,Arauca,...,,PRECIPITACION,PTPM_TT_D,Precipitación dia pluviométrico,Diaria,2000-01-05 00:00,0,50,,900\r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6968,37055010,AEROPUERTO SANTIAGO PÉREZ [37055010],7.0694444440,-70.73805556,128,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 08 - Santanderes-Arauca,Arauca,Arauca,...,,PRECIPITACION,PTPM_TT_D,Precipitación dia pluviométrico,Diaria,2019-01-29 00:00,0,50,,900\r
6969,37055010,AEROPUERTO SANTIAGO PÉREZ [37055010],7.0694444440,-70.73805556,128,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 08 - Santanderes-Arauca,Arauca,Arauca,...,,PRECIPITACION,PTPM_TT_D,Precipitación dia pluviométrico,Diaria,2019-01-30 00:00,0,50,,900\r
6970,37055010,AEROPUERTO SANTIAGO PÉREZ [37055010],7.0694444440,-70.73805556,128,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 08 - Santanderes-Arauca,Arauca,Arauca,...,,PRECIPITACION,PTPM_TT_D,Precipitación dia pluviométrico,Diaria,2019-01-31 00:00,0,50,,900\r
6971,37055010,AEROPUERTO SANTIAGO PÉREZ [37055010],7.0694444440,-70.73805556,128,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 08 - Santanderes-Arauca,Arauca,Arauca,...,,PRECIPITACION,PTPM_TT_D,Precipitación dia pluviométrico,Diaria,2019-02-01 00:00,0,50,,900\r


In [ ]:
"http://dhime.ideam.gov.co/arcgis/rest/services/CNE/Estaciones/MapServer/5/query?where=1%3D1&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=pjson"